<a href="https://colab.research.google.com/github/MiM0ulay/RiskMetrics/blob/main/RiskMetrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nasdaq-data-link


In [2]:
from datetime import date
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import nasdaqdatalink
import yfinance as yf

In [3]:
def calculate_risk_metric(ticker, start_date):
  print("   ")
  print(f"**** {ticker} RISK METRIC")
  # Download data
  df = yf.download(tickers=ticker, start=start_date, interval='1d')

  # Debugging: Check the column names

  # Reset the index and flatten column names (if needed)
  df.reset_index(inplace=True)

  # Handle multi-level columns (flatten them if they exist)
  df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]

  # Debugging: Check flattened column names

  # Rename columns for consistency
  if 'Open' in df.columns:
      df.rename(columns={'Date': 'date', 'Open': 'value'}, inplace=True)
  elif 'value' not in df.columns:
      raise KeyError("The required columns ('date', 'value') are not found in the DataFrame.")

  # Keep necessary columns
  df = df[['date', 'value']].copy()

  # Sort by date
  df.sort_values(by='date', inplace=True)

  # Calculate moving average
  moving_average_days = 365
  df['MA'] = df['value'].rolling(moving_average_days, min_periods=1).mean()

  # Ensure no NaN values
  df = df.dropna().reset_index(drop=True)

  # Calculate Preavg
  diminishing_factor = 0.395
  df['Preavg'] = (np.log(df['value']) - np.log(df['MA'])) * (np.arange(len(df)) + 1) ** diminishing_factor
  df['avg'] = (df['Preavg'] - df['Preavg'].cummin()) / (df['Preavg'].cummax() - df['Preavg'].cummin())
  price_per_risk = {
    round(risk, 1):round(np.exp(
        (risk * (df['Preavg'].cummax().iloc[-1] - (cummin := df['Preavg'].cummin().iloc[-1])) + cummin) / df.index[-1]**diminishing_factor + np.log(df['MA'].iloc[-1])
    ))
    for risk in np.arange(0.0, 1.0, 0.1)
  }
  # Print results
  print(df.tail(2))


In [4]:
cryptos = { 'ETH-USD': '2016-01-01', 'ETH-EUR': '2016-01-01', 'SOL-USD': '2020-01-01', 'SOL-EUR': '2020-01-01', 'BNB-USD': '2017-01-01' }

In [5]:
for ticker, start_date in cryptos.items():
  calculate_risk_metric(ticker, start_date)

   
**** ETH-USD RISK METRIC


[*********************100%***********************]  1 of 1 completed


           date        value           MA    Preavg       avg
2606 2024-12-28  3328.774658  3035.288776  2.063320  0.496859
2607 2024-12-29  3398.446289  3038.297123  2.504618  0.506304
   
**** ETH-EUR RISK METRIC


[*********************100%***********************]  1 of 1 completed


           date        value           MA    Preavg       avg
2604 2024-12-28  3196.219238  2804.929305  2.918468  0.502551
2605 2024-12-29  3258.963867  2808.048577  3.328599  0.511669
   
**** SOL-USD RISK METRIC


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

           date       value          MA    Preavg       avg
1723 2024-12-28  183.833389  154.616431  3.286134  0.579816
1724 2024-12-29  195.021194  154.859507  4.378971  0.599918
   
**** SOL-EUR RISK METRIC


           date       value          MA    Preavg       avg
1723 2024-12-28  180.597931  142.892610  4.446125  0.594048
1724 2024-12-29  187.016953  143.152322  5.075916  0.605781
   
**** BNB-USD RISK METRIC


[*********************100%***********************]  1 of 1 completed

           date       value          MA    Preavg       avg
2606 2024-12-28  690.429382  539.615354  5.509562  0.359192
2607 2024-12-29  720.953613  540.730729  6.431481  0.376083
